 Zero-shot classification pipeline

In [ ]:
# !pip install transformers
from transformers import pipeline
zero_shot_classifier = pipeline('zero-shot-classification', model="MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli", device=0)
sequence = "A new moon has been discovered in Jupiter's orbit."
class_names = ["the world", "sports", "business", "science/tech"]
zero_shot_classifier(sequence, class_names)

{'sequence': "A new moon has been discovered in Jupiter's orbit.",
 'labels': ['science/tech', 'the world', 'business', 'sports'],
 'scores': [0.9910086393356323,
  0.004674587398767471,
  0.003033342072740197,
  0.0012834330555051565]}

Preprocessing supervised dataset

In [ ]:
from numpy import testing
# from datasets import load_dataset
import pandas as pd
from numpy.random import RandomState
rng = RandomState()

data = pd.read_csv("C://Users//Phani//Desktop//fake_job_postings.csv")
del data['fraudulent']
del data['required_education']
del data['company_profile']
del data['requirements']
del data['benefits']
del data['telecommuting']
del data['has_company_logo']
del data['has_questions']	
del data['function']
del data['job_id']
del data['required_experience']
del data['department']
del data['industry']
data.reset_index(drop=True)

data.rename(columns={'title': 'job_title', 'salary_range' : 'salary'},inplace=True, errors='raise')

data_melted = pd.melt(data) 
data_melted.reset_index(drop=True)
data_melted.rename(columns={'variable': 'label', 'value' : 'text'},inplace=True, errors='raise')


from numpy.random import RandomState
rng = RandomState()
train = data_melted.sample(frac=0.9, random_state=rng)
test = data_melted.loc[~data_melted.index.isin(train.index)]
train = train.dropna()
test = test.dropna()
train.head(20)

label                                               text
22248         location                                           US, AL, 
75304  employment_type                                          Full-time
85131  employment_type                                          Full-time
18246         location                                    US, IL, Chicago
70203      description  Seeking a candidate to work full time in a pro...
8602         job_title  We are recruiting fragrance and beauty promoti...
7238         job_title                              Security Officer (SO)
28857         location                                    US, TX, El Paso
54710      description  We’re looking for an experienced Ruby develope...
25599         location                                    US, NY, Buffalo
4510         job_title          Russian-Ukrainian Social Media Specialist
20463         location                                      US, KS, Esbon
56878      description  MenuPad is a fast growing startup revolutioniz...
41547           salary                                        40000-60000
87613  employment_type                                          Full-time
78877  employment_type                                          Full-time
55647      description  VACANCY:  Support Worker (Personal Assistant) ...
9643         job_title                             Site Solution Designer
64013      description  Tel Tech Networks Inc. is currently seeking a ...
13923        job_title                              Application Developer

In [17]:

class_names = ["job title", "job salary", "job location", "job description", "department" ]


The other thing the script needs is the names of the classes. We'll put these into their own newline-delimitted `txt` as well:

In [ ]:
with open("job_posts/class_names.txt", 'w') as f:
    for label in class_names:
        f.write(label + '\n')

In [ ]:

!mkdir job_posts
with open("job_posts/train_unlabeled.txt", 'w') as f:
    for seq in train['text']:
      if len(seq)>512:
        seq=seq[:512]
        f.write(seq + '\n')
      else:  # print(seq + '\n')
        f.write(seq + '\n')

Training and distribution of the Model

In [ ]:
!python distill_classifier.py --fp16 --num_train_epochs 5 \
--data_file ./job_posts/train_unlabeled.txt \
--class_names_file ./job_posts/class_names.txt \
--hypothesis_template "This text is about {}." \
--student_name_or_path distilbert-base-uncased \
--output_dir ./C://Users//Phani//Desktop//model 2 --overwrite_output_dir

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
Moving 0 files to the new cache system
0it [00:00, ?it/s]
INFO:__main__:Training/evaluation parameters DistillTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
f

Load Distilled student model:

In [14]:
import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TextClassificationPipeline
tokenizer = AutoTokenizer.from_pretrained("C://Users//Phani//Desktop//model 2")
model = AutoModelForSequenceClassification.from_pretrained("C://Users//Phani//Desktop//model 2")
model.config

DistilBertConfig {
  "_name_or_path": "C://Users//Phani//Desktop//model 2",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "job title",
    "1": "job location",
    "2": "salary",
    "3": "job type",
    "4": "job description"
  },
  "initializer_range": 0.02,
  "label2id": {
    "job description": 4,
    "job location": 1,
    "job title": 0,
    "job type": 3,
    "salary": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.21.2",
  "vocab_size": 30522
}

Testing Accuracy of the trained model

In [ ]:


test['label'].replace({'description': 4, 'employment_type': 3,'job_title':0, 'location':1, 'salary':2 }, inplace = True) 
# print(test)
distilled_classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer,top_k=1, device=0) # return_all_scores=True,

            

import numpy as np
from time import time
from tqdm.auto import tqdm
start = time()
batch_size = 128 # larger batch size bc distilled model is more memory efficient
distilled_classifier.return_all_scores = False
preds = []
for i in tqdm(range(0, len(test), batch_size)):
    examples = test[i:i+batch_size]['text']
    outputs = distilled_classifier(examples)
    preds += [class_names.index(o['label']) for o in outputs]
accuracy = np.mean(np.array(preds) == np.array(test['label']))
print(f"Distilled model accuracy: {accuracy*100:0.2f}%")
print(f"Runtime: {time() - start : 0.2f} seconds")

Distilled model accuracy: 89.2%
Runtime: 4.132 seconds


Testing Classification on a random joblisting page

In [189]:
# %pip install BeautifulSoup4
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import re

# import torch
# !nvidia-smi
# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#   print('Not connected to a GPU')
# else:
#   print(gpu_info)

url = "https://www.able-group.co.uk/careers/"

r = requests.get(url)
soup = BeautifulSoup(r.content, "html.parser", fromEncoding = 'utf-8')
body = soup.find('body')

for tag in body.find_all(text=True):
    if tag.parent.name in ['style', 'script','noscript','[document]', 'head', 'title']:
        continue
       
    data = (str(tag))
    # print(data)
    sequences = [f"{data}"]
    sequences = [x for x in sequences if x != "\n"]
    sequences = [x for x in sequences if x != "\t"]
    sequences = [x for x in sequences if x != ""]
    sequences = [x for x in sequences if x != " "]
    sequences = [x for x in sequences if re.sub("[;:,.!()]", "", x).lower() not in ['job','job title', 'job description', 'description', 'job salary', 'salary', 'job type', 'type', 'job location','location', 'title']]
    # sequences = [x.lower() not in ['job', 'description', 'salary', 'type', 'location']  for x in sequences if x = re.sub("[;:,.!()]", "", x)]

    
    # print(sequences)
    distilled_classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer,top_k=1, device=0) # return_all_scores=True,
    # for _ in range(len(sequences)):
      # print(sequence)
      
    classified_d = pd.DataFrame({'tag_parent': tag.parent.name, 'sequence' :sequences, 'classified' :distilled_classifier(sequences)})
    print(classified_d)
    # classified_d.to_csv('C://Users//Phani//Desktop//classified_data.csv', header=False,   index=False) #mode='a',
       

    df = pd.DataFrame(classified_d)
    df = df.explode('sequence')
    df = df.explode('classified')
    df = df.fillna('')
    


    df['label'] = df['classified'].apply(f)
    df['score'] = df['classified'].apply(g)

    del df['classified']
    

    df = df.explode('label')
    df = df.explode('score')

    # df.head(10)
    


    job_title = df.loc[(df['label'] == 'job title')& (df['tag_parent'] != 'option') & (df['score'] > 0.3)].groupby('tag_parent').max().sort_values(by=['score', 'tag_parent'],ascending=False)
    # print(job_title)
    job_title =  job_title.reset_index()


    job_salary = df.loc[(df['label'] == 'salary')&(df['tag_parent'] != 'option')& (df['score'] > 0.3)].groupby('tag_parent').max().sort_values(by=['score', 'tag_parent'],ascending=False)
    job_salary =  job_salary.reset_index()
    # print(job_salary)

    job_location = df.loc[(df['label'] == 'job location')&(df['tag_parent'] != 'option')&(df['score'] > 0.3) ].groupby('tag_parent').max().sort_values(by=['score', 'tag_parent'],ascending=False)
    job_location =  job_location.reset_index()

    job_type= df.loc[(df['label'] == 'job type')&(df['tag_parent'] != 'option')& (df['score'] > 0.3)].groupby('tag_parent').max().sort_values(by=['score', 'tag_parent'],ascending=False)
    # print(job_type)
    job_type=  job_type.reset_index()
    # print(job_type)


    job_description = df.loc[(df['label'] == 'job description')&(df['tag_parent'] != 'option')& (df['score'] > 0.4)].groupby('tag_parent').max().sort_values(by=['score', 'tag_parent'],ascending=False)
    job_description =  job_description.reset_index()




    try:
        path_selector = body.find(lambda selector:selector.name==job_title.iloc[0,0] and job_title.iloc[0,1] in selector.text)
    except:
        path_selector = None
    try:
        path_selector2 = body.find(lambda selector:selector.name==job_location.iloc[0,0] and job_location.iloc[0,1] in selector.text)
    except:
        path_selector2 = None    
    try:
        path_selector3 = body.find(lambda selector:selector.name==job_salary.iloc[0,0] and job_salary.iloc[0,1] in selector.text)
    except:
        path_selector3 = None
    try:
        path_selector4 = body.find(lambda selector:selector.name==job_type.iloc[0,1] and job_type.iloc[0,2] in selector.text)
        # print(path_selector4)
    except:
        path_selector4 = None
    try:
        path_selector5 = body.find(lambda selector:selector.name==job_description.iloc[0,0] and job_description.iloc[0,1] in selector.text)
    except:
        path_selector5 = None


    
# # print(df.head(10))

def f(x):
    return [i['label'] for i in x]
def g(x):
    return [i['score'] for i in x]        
def titles():
    titles = []
    titles_links = []
    if path_selector and path_selector.parent.name  and path_selector.parent.attrs:
        
        # print(path_selector.parent.attrs)
        job_titles = body.find_all(path_selector.parent.name,attrs= path_selector.parent.attrs)
        # print(job_titles)
        for item in job_titles:
            try:
                titles.append(item.text)
            except:
                pass    
            for links in item:
                try:
                    titles_links = links['href']
                except:
                    pass
    elif path_selector:
        job_titles = body.find_all(path_selector.name,attrs= path_selector.attrs)
        
        for item in job_titles:
            try:
                titles = item.text
            except:
                pass    
            for links in item:
                try:
                    titles_links = links['href']
                except:
                    pass               
    else:
        titles = ''
        titles_links = ''
    return titles, titles_links         

def locations():
    locations = []             
    if path_selector2 and path_selector2.parent.name and path_selector2.parent.attrs:
        
        
        job_localtions = body.find_all(path_selector2.parent.name,attrs= path_selector2.parent.attrs)
        
        for item in job_localtions:
            try:
                locations = item.text
            except:
                pass    
    elif path_selector2:
        job_localtions = body.find_all(path_selector2.name,attrs= path_selector2.attrs)
        # print(job_titles)
        for item in job_localtions:
            try:
                locations = item.text
            except:
                pass
    else:
        locations = '' 
    return locations

def salaries():
    salaries = []
    if path_selector3 and path_selector3.parent.name and path_selector3.parent.attrs:
        
        # print(path_selector.parent.attrs)
        job_salaries = body.find_all(path_selector3.parent.name,attrs= path_selector3.parent.attrs)
        # print(job_titles)
        for item in job_salaries:
            try:
                salaries = item.text
            except:
                pass    
    elif path_selector3:
        job_salaries = body.find_all(path_selector3.name,attrs= path_selector3.attrs)
        # print(job_titles)
        for item in job_salaries:
            try:
                salaries = item.text
            except:
                pass 
    else:
        salaries =''    
    return salaries

def types():
    types   = []
    if path_selector4 and path_selector4.parent.name and path_selector4.parent.attrs:
        
        # print(path_selector.parent.attrs)
        job_types = body.find_all(path_selector4.parent.name,attrs= path_selector4.parent.attrs)
        # print(job_titles)
        for item in job_types:
            try:
                types = item.text
            except:
                pass    
    elif path_selector4:
        job_types = body.find_all(path_selector4.name,attrs= path_selector4.attrs)
        # print(job_titles)
        for item in job_types:
            try:
                types = item.text
            except:
                pass 
    else:
        types = ''        
    return types

def descriptions():
    if path_selector5 and path_selector5.parent.name and path_selector5.parent.attrs:
        
        # print(path_selector.parent.attrs)
        job_descriptions = body.find_all(path_selector5.parent.name,attrs= path_selector5.parent.attrs)
        # print(job_titles)
        for item in job_descriptions:
            try:
                descriptions = item.text
            except:
                pass    
    elif path_selector5:
        job_descriptions = body.find_all(path_selector5.name,attrs= path_selector5.attrs)
        # print(job_titles)
        for item in job_descriptions:
            try:
                descriptions = item.text
            except:
                pass   
    else:
        descriptions = ''
    return descriptions
list = []
list.append({'job title': titles(), 'job salary': salaries(), 'job type': types(), 'job_description': descriptions(), 'job location': locations()}, ignore_index=True)
print(list)



{'job_title': 'Emergency Plumber - Hull', 'job_title_link': 'https://www.able-group.co.uk/careers/emergency-plumber-hull/', 'job_type': None}
{'job_title': 'Emergency Plumber - Crewe', 'job_title_link': 'https://www.able-group.co.uk/careers/emergency-plumber-crewe/', 'job_type': None}
{'job_title': 'Sales Advisor', 'job_title_link': 'https://www.able-group.co.uk/careers/sales-advisor/', 'job_type': None}
{'job_title': 'Customer Service Co-ordinator', 'job_title_link': 'https://www.able-group.co.uk/careers/customer-service-co-ordinator/', 'job_type': None}
{'job_title': 'Emergency Plumber - Enfield', 'job_title_link': 'https://www.able-group.co.uk/careers/emergency-plumber-enfield/', 'job_type': None}
{'job_title': 'Emergency Gas Engineer - St Albans', 'job_title_link': 'https://www.able-group.co.uk/careers/emergency-gas-engineer-st-albans/', 'job_type': None}
{'job_title': 'Emergency Gas Engineer - Enfield', 'job_title_link': 'https://www.able-group.co.uk/careers/emergency-gas-engineer

Further Testing on 200 random job listing pages

In [11]:
import concurrent.futures
from multiprocessing.dummy import Pool as ThreadPool
from concurrent.futures import ThreadPoolExecutor
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36.',
# # "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "Cache-Control": "max-age=0",}

def scrape(index, row):
    try:
        # if row['career_pages'] != "": 
        response = requests.get(row['final_pages'], headers=headers , timeout=3)
        if response.status_code <= 400:
            
            soup = BeautifulSoup(response.content, "html.parser", Encoding = 'utf-8')
            body = soup.find('body')
        classified_dict = {}
        classified_d = []    
        for tag in body.find_all(text=True):
            # if tag.parent.name in ['style', 'script','noscript','[document]', 'head', 'title']:
                # continue
            
            data = (str(tag))
            # print(data)
            sequences = [f"{data}"]
            sequences = [x for x in sequences if x != "\n"]
            sequences = [x for x in sequences if x != "\t"]
            sequences = [x for x in sequences if x != ""]
            sequences = [x for x in sequences if x != " "]
            sequences = [x for x in sequences if re.sub("[;:,.!()]", "", x).lower() not in ['job','job title', 'job description', 'description', 'job salary', 'salary', 'job type', 'type', 'job location','location', 'title']]
            sequences = [x.lower() not in ['job', 'description', 'salary', 'type', 'location']  for x in sequences if x = re.sub("[;:,.!()]", "", x)]
            df5.loc[index, 'classified'] = [f"{data}"]
            
            print(sequences)
            distilled_classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer,top_k=1, device=0) # return_all_scores=True,
            # for _ in range(len(sequences)):
            # print(sequence)
            
            classified_dict = ({'tag_parent': tag.parent.name, 'sequence' :sequences, 'classified' :distilled_classifier(sequences)})
            classified_d.append(classified_dict)
           
        

        df = pd.DataFrame(classified_d)
        df = df.explode('sequence')
        df = df.explode('classified')
        df = df.fillna('')
        


        df['label'] = df['classified'].apply(f)
        df['score'] = df['classified'].apply(g)

        del df['classified']
        

        df = df.explode('label')
        df = df.explode('score')

        # df.head(10)
       


        job_title = df.loc[(df['label'] == 'job title')& (df['tag_parent'] != 'option') & (df['score'] > 0.3)].groupby('tag_parent').max().sort_values(by=['score', 'tag_parent'],ascending=False)
        # print(job_title)
        job_title =  job_title.reset_index()


        job_salary = df.loc[(df['label'] == 'salary')&(df['tag_parent'] != 'option')& (df['score'] > 0.3)].groupby('tag_parent').max().sort_values(by=['score', 'tag_parent'],ascending=False)
        job_salary =  job_salary.reset_index()
        # print(job_salary)

        job_location = df.loc[(df['label'] == 'job location')&(df['tag_parent'] != 'option')&(df['score'] > 0.3) ].groupby('tag_parent').max().sort_values(by=['score', 'tag_parent'],ascending=False)
        job_location =  job_location.reset_index()

        job_type= df.loc[(df['label'] == 'job type')&(df['tag_parent'] != 'option')& (df['score'] > 0.3)].groupby('tag_parent').max().sort_values(by=['score', 'tag_parent'],ascending=False)
        # print(job_type)
        job_type=  job_type.reset_index()
        # print(job_type)


        job_description = df.loc[(df['label'] == 'job description')&(df['tag_parent'] != 'option')& (df['score'] > 0.4)].groupby('tag_parent').max().sort_values(by=['score', 'tag_parent'],ascending=False)
        job_description =  job_description.reset_index()




        try:
            path_selector = body.find(lambda selector:selector.name==job_title.iloc[0,0] and job_title.iloc[0,1] in selector.text)
        except:
            path_selector = None
        try:
            path_selector2 = body.find(lambda selector:selector.name==job_location.iloc[0,0] and job_location.iloc[0,1] in selector.text)
        except:
            path_selector2 = None    
        try:
            path_selector3 = body.find(lambda selector:selector.name==job_salary.iloc[0,0] and job_salary.iloc[0,1] in selector.text)
        except:
            path_selector3 = None
        try:
            path_selector4 = body.find(lambda selector:selector.name==job_type.iloc[0,1] and job_type.iloc[0,2] in selector.text)
            # print(path_selector4)
        except:
            path_selector4 = None
        try:
            path_selector5 = body.find(lambda selector:selector.name==job_description.iloc[0,0] and job_description.iloc[0,1] in selector.text)
        except:
            path_selector5 = None

   
        list = []
        list.append({'job title': titles(), 'job salary': salaries(), 'job type': types(), 'job_description': descriptions(), 'job location': locations()}, ignore_index=True)
        print(list)
        df5.loc[index, 'classified'] = list
# # print(df.head(10))
    except:
        pass
def f(x):
    return [i['label'] for i in x]
def g(x):
    return [i['score'] for i in x]        
def titles():
    titles = []
    titles_links = []
    if path_selector and path_selector.parent.name  and path_selector.parent.attrs:
        
        # print(path_selector.parent.attrs)
        job_titles = body.find_all(path_selector.parent.name,attrs= path_selector.parent.attrs)
        # print(job_titles)
        for item in job_titles:
            try:
                titles.append(item.text)
            except:
                pass    
            for links in item:
                try:
                    titles_links = links['href']
                except:
                    pass
    elif path_selector:
        job_titles = body.find_all(path_selector.name,attrs= path_selector.attrs)
        
        for item in job_titles:
            try:
                titles = item.text
            except:
                pass    
            for links in item:
                try:
                    titles_links = links['href']
                except:
                    pass               
    else:
        titles = ''
        titles_links = ''
    return titles, titles_links         

def locations():
    locations = []             
    if path_selector2 and path_selector2.parent.name and path_selector2.parent.attrs:
        
        
        job_localtions = body.find_all(path_selector2.parent.name,attrs= path_selector2.parent.attrs)
        
        for item in job_localtions:
            try:
                locations = item.text
            except:
                pass    
    elif path_selector2:
        job_localtions = body.find_all(path_selector2.name,attrs= path_selector2.attrs)
        # print(job_titles)
        for item in job_localtions:
            try:
                locations = item.text
            except:
                pass
    else:
        locations = '' 
    return locations

def salaries():
    salaries = []
    if path_selector3 and path_selector3.parent.name and path_selector3.parent.attrs:
        
        # print(path_selector.parent.attrs)
        job_salaries = body.find_all(path_selector3.parent.name,attrs= path_selector3.parent.attrs)
        # print(job_titles)
        for item in job_salaries:
            try:
                salaries = item.text
            except:
                pass    
    elif path_selector3:
        job_salaries = body.find_all(path_selector3.name,attrs= path_selector3.attrs)
        # print(job_titles)
        for item in job_salaries:
            try:
                salaries = item.text
            except:
                pass 
    else:
        salaries =''    
    return salaries

def types():
    types   = []
    if path_selector4 and path_selector4.parent.name and path_selector4.parent.attrs:
        
        # print(path_selector.parent.attrs)
        job_types = body.find_all(path_selector4.parent.name,attrs= path_selector4.parent.attrs)
        # print(job_titles)
        for item in job_types:
            try:
                types = item.text
            except:
                pass    
    elif path_selector4:
        job_types = body.find_all(path_selector4.name,attrs= path_selector4.attrs)
        # print(job_titles)
        for item in job_types:
            try:
                types = item.text
            except:
                pass 
    else:
        types = ''        
    return types

def descriptions():
    if path_selector5 and path_selector5.parent.name and path_selector5.parent.attrs:
        
        # print(path_selector.parent.attrs)
        job_descriptions = body.find_all(path_selector5.parent.name,attrs= path_selector5.parent.attrs)
        # print(job_titles)
        for item in job_descriptions:
            try:
                descriptions = item.text
            except:
                pass    
    elif path_selector5:
        job_descriptions = body.find_all(path_selector5.name,attrs= path_selector5.attrs)
        # print(job_titles)
        for item in job_descriptions:
            try:
                descriptions = item.text
            except:
                pass   
    else:
        descriptions = ''
    return descriptions


df5 = pd.read_csv('c://users//phani//desktop//bior_companies6.csv', encoding='utf8').fillna("")
del df5['company_address']
del df5['company_website']
del df5['validated_link']
del df5['invalid_pages']
del df5['career_pages']
del df5['job_pages']
del df5['validated_career_pages']
del df5['validated_job_pages']
df5['classified'] = ''
# df5  = df5.loc[df5['final_pages'] != '']
# df5 =  df5.loc[:200]
df5 = df5[df5['final_pages'].notnull() & (df5['final_pages'].str.contains('vacancies') | df5['final_pages'].str.contains('jobs'))].sample(200)


with concurrent.futures.ThreadPoolExecutor(max_workers= 25) as executor:
    for index, row in df5.iterrows():
        executor.submit(scrape,index, row)
    
        # future1 =  executor.submit(scrape, index, row)
        # future2 = executor.submit(sort(), future1.results())

df5.to_csv('c://users//phani//desktop//results2.csv', index=False, encoding='utf8')


Cleaning and creating new dataframe with 200 samples from above

In [3]:
import pandas as pd
df5 = pd.read_csv('c://users//phani//desktop//bior_companies6.csv', encoding='utf8').fillna("")
del df5['company_address']
del df5['company_website']
del df5['validated_link']
del df5['invalid_pages']
del df5['career_pages']
del df5['job_pages']
del df5['validated_job_pages']
del df5['validated_career_pages']
# df5['classified'] = ''
# df5  = df5.loc[df5['final_pages'] != '']
# df5 =  df5.sample(n=200)
df5 = df5[df5['final_pages'].notnull() & (df5['final_pages'].str.contains('vacancies') | df5['final_pages'].str.contains('jobs'))].sample(200)
df5

company_name  \
210                      Alstom Group   
2940                        NBC Group   
3275     Phoenix Gas Services Limited   
2357                 Karro Food Group   
2626                 MacTaggart Scott   
...                               ...   
4108               The Woodland Trust   
2402              KINGSTON UNIVERSITY   
3173              Oxford City Council   
4163                           TORC24   
1810  GLL (Greenwich Leisure Limited)   

                                            final_pages  
210                       https://jobsearch.alstom.com/  
2940  https://www.nbc-group.co.uk/working-with-us/cu...  
3275  https://www.naturalhr.net/hr/recruitment/jobs/...  
2357  https://krb-jobs.brassring.com/TGnewUI/Search/...  
2626                http://www.mactag.com/job-vacancies  
...                                                 ...  
4108                 https://jobs.woodlandtrust.org.uk/  
2402  https://jobs.kingston.ac.uk/home.html#filter=p...  
3173          https://www.oxford.gov.uk/info/20014/jobs  
4163  http://www.connells.co.uk/about-us/jobs-in-lei...  
1810                     https://www.better.org.uk/jobs  

[200 rows x 2 columns]

Classification on 200 samples

In [ ]:
# %pip install BeautifulSoup4
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import re

# import torch
# !nvidia-smi
# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#   print('Not connected to a GPU')
# else:
#   print(gpu_info)

url = "https://www.ascenti.co.uk//careers/vacancies"

r = requests.get(url)
soup = BeautifulSoup(r.content, "html.parser", fromEncoding = 'utf-8')
body = soup.find('body')
results = {}
results2 = []
for tag in body.find_all(text=True):
    if tag.parent.name in ['style', 'script','noscript','[document]', 'head', 'title']:
        continue
       
    data = (str(tag))
    # print(data)
    sequences = [f"{data}"]
    sequences = [x for x in sequences if x != "\n"]
    sequences = [x for x in sequences if x != "\t"]
    sequences = [x for x in sequences if x != ""]
    sequences = [x for x in sequences if x != " "]
    sequences = [x for x in sequences if re.sub("[;:,.!()]", "", x).lower() not in ['job','job title', 'job description', 'description', 'job salary', 'salary', 'job type', 'type', 'job location','location', 'title']]
    # sequences = [x.lower() not in ['job', 'description', 'salary', 'type', 'location']  for x in sequences if x = re.sub("[;:,.!()]", "", x)]

    
    # print(sequences)
    distilled_classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer,top_k=1, device=0)
    
    
    results = ({'tag_parent': tag.parent.name, 'sequences': sequences, 'classified': distilled_classifier(sequences)})
    # for _ in range(len(test)):
    # test = [{'parent_name': tag.parent.name, 'sequences': sequences, 'classifier': distilled_classifier(sequences)}]  
    results2.append(results)
    # results2.insert(results)
    # for result in results:
    # df = pd.DataFrame()
    # df['results'] = results 

# print(results2)    
# from itertools import chain
df = pd.DataFrame(results2)
# # df.head(10)
import ast
df = df.explode('sequences')
df = df.explode('classified')
df = df.fillna('')
# df['classified'] = df['classified'].apply(ast.literal_eval)

# df.to_csv('C://Users//Phani//Desktop//classified_data.csv', index = False)


df['label'] = df['classified'].apply(lambda x: [i['label'] for i in x])
df['score'] = df['classified'].apply(lambda x: [i['score'] for i in x])

del df['classified']
# df.head(10)

df = df.explode('label')
df = df.explode('score')

df.head(10)

job_title = df.loc[(df['label'] == 'job title')& (df['tag_parent'] != 'option') & (df['score'] > 0.3)].groupby('tag_parent').max().sort_values(by=['score', 'tag_parent'],ascending=False)
# print(job_title)
job_title =  job_title.reset_index()


job_salary = df.loc[(df['label'] == 'salary')&(df['tag_parent'] != 'option')& (df['score'] > 0.3)].groupby('tag_parent').max().sort_values(by=['score', 'tag_parent'],ascending=False)
job_salary =  job_salary.reset_index()
# print(job_salary)

job_location = df.loc[(df['label'] == 'job location')&(df['tag_parent'] != 'option')&(df['score'] > 0.3) ].groupby('tag_parent').max().sort_values(by=['score', 'tag_parent'],ascending=False)
job_location =  job_location.reset_index()

job_type= df.loc[(df['label'] == 'job type')&(df['tag_parent'] != 'option')& (df['score'] > 0.3)].groupby('tag_parent').max().sort_values(by=['score', 'tag_parent'],ascending=False)
# print(job_type)
job_type=  job_type.reset_index()
# print(job_type)


job_description = df.loc[(df['label'] == 'job description')&(df['tag_parent'] != 'option')& (df['score'] > 0.4)].groupby('tag_parent').max().sort_values(by=['score', 'tag_parent'],ascending=False)
job_description =  job_description.reset_index()




try:
    path_selector = body.find(lambda selector:selector.name==job_title.iloc[0,0] and job_title.iloc[0,1] in selector.text)
except:
    path_selector = None
try:
    path_selector2 = body.find(lambda selector:selector.name==job_location.iloc[0,0] and job_location.iloc[0,1] in selector.text)
except:
    path_selector2 = None    
try:
    path_selector3 = body.find(lambda selector:selector.name==job_salary.iloc[0,0] and job_salary.iloc[0,1] in selector.text)
except:
    path_selector3 = None
try:
    path_selector4 = body.find(lambda selector:selector.name==job_type.iloc[0,1] and job_type.iloc[0,2] in selector.text)
    # print(path_selector4)
except:
    path_selector4 = None
try:
    path_selector5 = body.find(lambda selector:selector.name==job_description.iloc[0,0] and job_description.iloc[0,1] in selector.text)
except:
    path_selector5 = None
# print(path_selector3)
def titles():
    titles = []
    titles_links = []
    if path_selector and path_selector.parent.name  and path_selector.parent.attrs:
        
        # print(path_selector.parent.attrs)
        job_titles = body.find_all(path_selector.parent.name,attrs= path_selector.parent.attrs)
        # print(job_titles)
        for item in job_titles:
            try:
                titles.append(item.text)
            except:
                pass    
            for links in item:
                try:
                    titles_links = links['href']
                except:
                    pass
    elif path_selector:
        job_titles = body.find_all(path_selector.name,attrs= path_selector.attrs)
        
        for item in job_titles:
            try:
                titles = item.text
            except:
                pass    
            for links in item:
                try:
                    titles_links = links['href']
                except:
                    pass               
    else:
        titles = ''
        titles_links = ''
    return titles, titles_links         

def locations():
    locations = []             
    if path_selector2 and path_selector2.parent.name and path_selector2.parent.attrs:
        
        
        job_localtions = body.find_all(path_selector2.parent.name,attrs= path_selector2.parent.attrs)
        
        for item in job_localtions:
            try:
                locations = item.text
            except:
                pass    
    elif path_selector2:
        job_localtions = body.find_all(path_selector2.name,attrs= path_selector2.attrs)
        # print(job_titles)
        for item in job_localtions:
            try:
                locations = item.text
            except:
                pass
    else:
        locations = '' 
    return locations

def salaries():
    salaries = []
    if path_selector3 and path_selector3.parent.name and path_selector3.parent.attrs:
        
        # print(path_selector.parent.attrs)
        job_salaries = body.find_all(path_selector3.parent.name,attrs= path_selector3.parent.attrs)
        # print(job_titles)
        for item in job_salaries:
            try:
                salaries = item.text
            except:
                pass    
    elif path_selector3:
        job_salaries = body.find_all(path_selector3.name,attrs= path_selector3.attrs)
        # print(job_titles)
        for item in job_salaries:
            try:
                salaries = item.text
            except:
                pass 
    else:
        salaries =''    
    return salaries

def types():
    types   = []
    if path_selector4 and path_selector4.parent.name and path_selector4.parent.attrs:
        
        # print(path_selector.parent.attrs)
        job_types = body.find_all(path_selector4.parent.name,attrs= path_selector4.parent.attrs)
        # print(job_titles)
        for item in job_types:
            try:
                types = item.text
            except:
                pass    
    elif path_selector4:
        job_types = body.find_all(path_selector4.name,attrs= path_selector4.attrs)
        # print(job_titles)
        for item in job_types:
            try:
                types = item.text
            except:
                pass 
    else:
        types = ''        
    return types

def descriptions():
    if path_selector5 and path_selector5.parent.name and path_selector5.parent.attrs:
        
        # print(path_selector.parent.attrs)
        job_descriptions = body.find_all(path_selector5.parent.name,attrs= path_selector5.parent.attrs)
        # print(job_titles)
        for item in job_descriptions:
            try:
                descriptions = item.text
            except:
                pass    
    elif path_selector5:
        job_descriptions = body.find_all(path_selector5.name,attrs= path_selector5.attrs)
        # print(job_titles)
        for item in job_descriptions:
            try:
                descriptions = item.text
            except:
                pass   
    else:
        descriptions = ''
    return descriptions

df2 = pd.DataFrame()


df2 = df2.append({'job title': titles(), 'job salary': salaries(), 'job type': types(), 'job_description': descriptions(), 'job location': locations()}, ignore_index=True)
# types()
# df2

Scraping validation data from source cv-libary for validation

In [5]:
df6 = df5
# urls = []
# df6['link'] = ''
df6['link'] = 'https://www.cv-library.co.uk/companies/employers?q=' + df6['company_name']
df6['link'] = df6['link'].str.replace(' ','%20')
del df6['classified']
# print(url)
# for url in urls:
    # df6.loc['link'] = url
df6.to_csv('c://users//phani//desktop//testing.csv', index=False, encoding='utf8')
df6

company_name  \
2132                              Icosa   
3576                Ripley Court School   
1703                    FRED PERRY Ltd.   
32                A.S. Matthews Limited   
1260                               DFDS   
...                                 ...   
2608                Lyreco UK & Ireland   
1939                   Haringey Council   
1306                     Doosan Babcock   
3679                      Safestore Ltd   
1374  East Anglian Air Ambulance (EAAA)   

                                            final_pages  \
2132            https://www.icosawater.co.uk/vacancies/   
3576  https://www.ripleycourt.co.uk/16/current-vacan...   
1703                 https://careers.fredperry.com/jobs   
32                  https://www.kormornicks.co.uk/jobs/   
1260    https://www.dfds.com/en/about/careers/vacancies   
...                                                 ...   
2608  https://lyreco.wd3.myworkdayjobs.com/en-US/Lyr...   
1939             https://jobs.haringey.gov.uk/vacancies   
1306  https://www.doosanbabcock.com/en/career/vacanc...   
3679              https://jobs.safestore.co.uk/careers/   
1374          http://www.eaaa.org.uk/careers/vacancies/   

                                                   link  \
2132  https://www.cv-library.co.uk/companies/employe...   
3576  https://www.cv-library.co.uk/companies/employe...   
1703  https://www.cv-library.co.uk/companies/employe...   
32    https://www.cv-library.co.uk/companies/employe...   
1260  https://www.cv-library.co.uk/companies/employe...   
...                                                 ...   
2608  https://www.cv-library.co.uk/companies/employe...   
1939  https://www.cv-library.co.uk/companies/employe...   
1306  https://www.cv-library.co.uk/companies/employe...   
3679  https://www.cv-library.co.uk/companies/employe...   
1374  https://www.cv-library.co.uk/companies/employe...   

                                                  link2  
2132  https://www.cv-library.co.uk/list-jobs/378351/...  
3576  https://www.cv-library.co.uk/list-jobs/377962/...  
1703  https://www.cv-library.co.uk/list-jobs/353302/...  
32                                                       
1260  https://www.cv-library.co.uk/list-jobs/320862/...  
...                                                 ...  
2608  https://www.cv-library.co.uk/list-jobs/367637/...  
1939  https://www.cv-library.co.uk/list-jobs/373856/...  
1306  https://www.cv-library.co.uk/list-jobs/310721/...  
3679  https://www.cv-library.co.uk/list-jobs/212081/...  
1374                                                     

[200 rows x 4 columns]

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
import concurrent.futures

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36'}


def scrape(index, row):
    
    try:
        
        response = requests.get(row['link'], headers=headers , timeout=3)
        if response.status_code <= 400:
        # print(response.status_code)
        # if response.elapsed.total_seconds() > 3:
        # #     print('The response is taking too long')
        # # # elif response.status_code <= 400 or response.status_code == "":
            soup = BeautifulSoup(response.content, 'html.parser')
        #     # # listed = []
            links = soup.find_all("div", {"class": "companies-results-company__title"})

            for link in links:
                if link.find("a").text == row['company_name']:
                    # print(link.find("a")["href"])
                    df.loc[index, 'link2'] = 'https://www.cv-library.co.uk' + (link.find("a")["href"])
                

    except:
      print("invalid url")

# async def scrap(row):
#     await asyncio.sleep(0)
#     scrape(row)

# if __name__ == '__main__':
df = df6
df['link2'] = ''

with concurrent.futures.ThreadPoolExecutor(max_workers= 25) as executor:
    for index, row in df.iterrows():
        executor.submit(scrape, index, row)
        # scrape(index, row)



       
df.to_csv('c://users//phani//desktop//testing2.csv', index=False, encoding='utf8')
df

company_name  \
2132                              Icosa   
3576                Ripley Court School   
1703                    FRED PERRY Ltd.   
32                A.S. Matthews Limited   
1260                               DFDS   
...                                 ...   
2608                Lyreco UK & Ireland   
1939                   Haringey Council   
1306                     Doosan Babcock   
3679                      Safestore Ltd   
1374  East Anglian Air Ambulance (EAAA)   

                                            final_pages  \
2132            https://www.icosawater.co.uk/vacancies/   
3576  https://www.ripleycourt.co.uk/16/current-vacan...   
1703                 https://careers.fredperry.com/jobs   
32                  https://www.kormornicks.co.uk/jobs/   
1260    https://www.dfds.com/en/about/careers/vacancies   
...                                                 ...   
2608  https://lyreco.wd3.myworkdayjobs.com/en-US/Lyr...   
1939             https://jobs.haringey.gov.uk/vacancies   
1306  https://www.doosanbabcock.com/en/career/vacanc...   
3679              https://jobs.safestore.co.uk/careers/   
1374          http://www.eaaa.org.uk/careers/vacancies/   

                                                   link  \
2132  https://www.cv-library.co.uk/companies/employe...   
3576  https://www.cv-library.co.uk/companies/employe...   
1703  https://www.cv-library.co.uk/companies/employe...   
32    https://www.cv-library.co.uk/companies/employe...   
1260  https://www.cv-library.co.uk/companies/employe...   
...                                                 ...   
2608  https://www.cv-library.co.uk/companies/employe...   
1939  https://www.cv-library.co.uk/companies/employe...   
1306  https://www.cv-library.co.uk/companies/employe...   
3679  https://www.cv-library.co.uk/companies/employe...   
1374  https://www.cv-library.co.uk/companies/employe...   

                                                  link2  
2132  https://www.cv-library.co.uk/list-jobs/378351/...  
3576  https://www.cv-library.co.uk/list-jobs/377962/...  
1703  https://www.cv-library.co.uk/list-jobs/353302/...  
32                                                       
1260  https://www.cv-library.co.uk/list-jobs/320862/...  
...                                                 ...  
2608  https://www.cv-library.co.uk/list-jobs/367637/...  
1939  https://www.cv-library.co.uk/list-jobs/373856/...  
1306  https://www.cv-library.co.uk/list-jobs/310721/...  
3679  https://www.cv-library.co.uk/list-jobs/212081/...  
1374                                                     

[200 rows x 4 columns]

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
import concurrent.futures

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36'}


def scrape(index, row):
    
    try:
        # print(row['link2'])
        response = requests.get(row['link2'], headers=headers , timeout=3)
        # if response.status_code <= 400:
        # # print(response.status_code)
        # # if response.elapsed.total_seconds() > 3:
        # # #     print('The response is taking too long')
        # # # # elif response.status_code <= 400 or response.status_code == "":
        soup = BeautifulSoup(response.content, 'html.parser')
        results = soup.find('div', id='profile-list-jobs')

        job_elems = results.find_all('article', class_='job search-card')
        # print(job_elems)
        d =  []
        for job_elem in job_elems:
            title_elem = job_elem.find('h2', class_='job__title')
            location_elem = job_elem.find('span', class_='job__details-location')
            salary_elem = job_elem.find('dd', class_='job__details-value salary')
            jobtype_elem = job_elem.find('dt', attrs = {'class':'job__details-term job-type'}).find_next_sibling()
            # print(jobtype_elem.text)

            d.append({'title':title_elem.text.strip(), 'location': location_elem.text.strip(), 'salary': salary_elem.text, 'jobtype':jobtype_elem.text})
            
        
        df.loc[index, 'data'] = d
            # if None in (title_elem, location_elem, salary_elem, jobtype_elem):
            #     continue
        # df.loc[index, 'title']=  d['title_elem']
        # df.loc[index, 'location'] = d['location_elem']
        # df.loc[index, 'job_type'] = d['salary_elem']
        # df.loc[index, 'job_salary'] = d['jobtype_elem']
    #   

    except:
      pass

# async def scrap(row):
#     await asyncio.sleep(0)
#     scrape(row)

# if __name__ == '__main__':
df['data'] = ''

with concurrent.futures.ThreadPoolExecutor(max_workers= 10) as executor:
    for index, row in df.iterrows():
        executor.submit(scrape, index, row)
        # scrape(index, row)




      
df.to_csv('c://users//phani//desktop//testing3.csv', index=False, encoding='utf8')
df

company_name  \
2132                              Icosa   
3576                Ripley Court School   
1703                    FRED PERRY Ltd.   
32                A.S. Matthews Limited   
1260                               DFDS   
...                                 ...   
2608                Lyreco UK & Ireland   
1939                   Haringey Council   
1306                     Doosan Babcock   
3679                      Safestore Ltd   
1374  East Anglian Air Ambulance (EAAA)   

                                            final_pages  \
2132            https://www.icosawater.co.uk/vacancies/   
3576  https://www.ripleycourt.co.uk/16/current-vacan...   
1703                 https://careers.fredperry.com/jobs   
32                  https://www.kormornicks.co.uk/jobs/   
1260    https://www.dfds.com/en/about/careers/vacancies   
...                                                 ...   
2608  https://lyreco.wd3.myworkdayjobs.com/en-US/Lyr...   
1939             https://jobs.haringey.gov.uk/vacancies   
1306  https://www.doosanbabcock.com/en/career/vacanc...   
3679              https://jobs.safestore.co.uk/careers/   
1374          http://www.eaaa.org.uk/careers/vacancies/   

                                                   link  \
2132  https://www.cv-library.co.uk/companies/employe...   
3576  https://www.cv-library.co.uk/companies/employe...   
1703  https://www.cv-library.co.uk/companies/employe...   
32    https://www.cv-library.co.uk/companies/employe...   
1260  https://www.cv-library.co.uk/companies/employe...   
...                                                 ...   
2608  https://www.cv-library.co.uk/companies/employe...   
1939  https://www.cv-library.co.uk/companies/employe...   
1306  https://www.cv-library.co.uk/companies/employe...   
3679  https://www.cv-library.co.uk/companies/employe...   
1374  https://www.cv-library.co.uk/companies/employe...   

                                                  link2  \
2132  https://www.cv-library.co.uk/list-jobs/378351/...   
3576  https://www.cv-library.co.uk/list-jobs/377962/...   
1703  https://www.cv-library.co.uk/list-jobs/353302/...   
32                                                        
1260  https://www.cv-library.co.uk/list-jobs/320862/...   
...                                                 ...   
2608  https://www.cv-library.co.uk/list-jobs/367637/...   
1939  https://www.cv-library.co.uk/list-jobs/373856/...   
1306  https://www.cv-library.co.uk/list-jobs/310721/...   
3679  https://www.cv-library.co.uk/list-jobs/212081/...   
1374                                                      

                                                   data  
2132                                                     
3576                                                     
1703                                                     
32                                                       
1260                                                     
...                                                 ...  
2608  [{'title': 'Field Sales Executive', 'location'...  
1939  [{'title': 'Education Welfare and Child Employ...  
1306                                                     
3679                                                     
1374                                                     

[200 rows x 5 columns]

Validation between scraped source data and ML classified data

In [99]:
df15 = pd.read_csv('c://users//phani//desktop//results.csv') 

def percent_match(df, df15):

    for i in range(len(df)): 
        for j in range(len(df15)):
            if df['data'] != '':
                if df15['classified'] != '':
                    if (df.loc['data']['title'][i] == df15['classified']['title'][j]) & (df.loc['data']['salary'][i] == df15['classified']['salary'][j]) & (df.loc['data']['jobtype'][i] == df15['classified']['jobtype'][j]) & (df.loc['location']['description'][i] == df15['classified']['product'][j]): 
                        count = count + 1
  
    print("Percentage of items that are matching:", (count/len(df.loc['data']))*100, "%")


Percentage of items that are matching: 42 %
